In [5]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

In [1]:
class Data:
    def __init__(self, path, name, wavelengths, c_type):
        f = pd.read_csv(path)
        wv_raw = np.array(f.wv)
        R = np.array(f.R)
        self.name = name
        self.df = pd.Series()
        self.f = interp1d(wv_raw, R, kind='cubic')
        self.min_wv = min(wv_raw)
        self.max_wv = max(wv_raw)
        self.df = pd.Series(data=R, index=wv_raw, name=name)
        self.interp(wavelengths)
        
    def interp(self, wavelengths):
        wavelengths = wavelengths[np.nonzero(np.logical_and(wavelengths > self.min_wv, wavelengths < self.max_wv))]
        data = self.f(wavelengths)
        self.df = pd.Series(data=data, index=wavelengths, name=self.name)
    
    def norm(self, wavelength):
        df = self.df.ix[wavelength]
        df = df - df.min()
        df = df / df.max()
        return df

In [62]:
data = Data('C:/WORK!!!!/VF088 - ARC Thickness/EKE-HC d.csv', 'Data', np.arange(300,1700), 'R')


In [44]:
z = 10
power = 3
deriv = 0

df = data.df
m = z*2+1

mirror = np.array(df.loc[df.index.min()+1:df.index.min()+1+z])[::-1]
pre = pd.Series(mirror, np.arange(df.index.min()-1-z, df.index.min()), name='Data')

mirror = np.array(df.loc[df.index.max()-z:df.index.max()-1])[::-1]
print(mirror)
app = pd.Series(mirror, np.arange(df.index.max()+1, df.index.max()+1+z), name='Data')
df = pd.concat([pre, df, app])

[ 0.62479731  0.62624661  0.62480601  0.62897326  0.6296635   0.62917771
  0.62809776  0.6267937   0.62725939  0.62835453]


331    0.678040
332    0.645101
333    0.652057
334    0.892480
335    0.835449
336    0.624655
337    0.583661
338    0.777643
339    0.857572
340    0.527690
341    0.581022
342    0.638365
343    0.581022
344    0.527690
345    0.857572
346    0.777643
347    0.583661
348    0.624655
349    0.835449
350    0.892480
351    0.652057
352    0.645101
353    0.678040
354    0.679424
355    0.702101
356    0.732835
357    0.735163
358    0.770052
359    0.729046
360    0.737625
         ...   
913    0.625646
914    0.625465
915    0.622660
916    0.622779
917    0.628249
918    0.630314
919    0.633398
920    0.629746
921    0.627813
922    0.628355
923    0.627259
924    0.626794
925    0.628098
926    0.629178
927    0.629664
928    0.628973
929    0.624806
930    0.626247
931    0.624797
932    0.619225
933    0.624797
934    0.626247
935    0.624806
936    0.628973
937    0.629664
938    0.629178
939    0.628098
940    0.626794
941    0.627259
942    0.628355
Name: Data, dtype: float

In [57]:
from matplotlib import pyplot as plt

In [66]:
arr = savitzky_golay(y=np.array(data.df), window_size=11, order=3, deriv=0, rate=1)
df = pd.Series(arr, data.df.index, name="S-G smoothed")
#plt.plot(df.index, df)
plt.plot(data.df.index, data.df)
plt.show()

In [ ]:
data = np.arange(1, 26)
index = np.arange(1, 6, .2).round(1)

x = pd.DataFrame(np.array([data, index.astype(int)]).T, columns=['data', 'index'])
x.mean(1)

In [ ]:
x = 3
y = -4

np.arctan(y/x)*180/np.pi